In [2]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --verify-certificate False --cluster-id j-1YS0E8OAQ9TGP --auth-type None

Successfully read emr cluster(j-1YS0E8OAQ9TGP) details
Initiating EMR connection..
Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
{"namespace": "sagemaker-analytics", "cluster_id": "j-1YS0E8OAQ9TGP", "error_message": null, "success": true, "service": "emr", "operation": "connect"}


In [3]:
import pyspark
from functools import reduce
from operator import add
from pytz import timezone
from pyspark import  StorageLevel, SparkContext, SparkConf
import pyspark.sql.types  as st
import pyspark.sql.functions  as func
from pyspark.sql.functions import *
from pyspark.sql.window import Window as W
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from datetime import datetime, timedelta
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

conf = SparkConf()
spark_cfgs = {
    "spark.app.name": "NVS ETL",
    "spark.hadoop.hive.exec.dynamic.partition.mode": "nonstrict",
    "spark.spark.sql.legacy.timeParserPolicy": "LEGACY",
    "spark.sql.parquet.compression.codec": "SNAPPY",
    "spark.sql.sources.partitionOverwriteMode": "dynamic",
    "spark.sql.hive.convertMetastoreParquet": False
}

for k, v in spark_cfgs.items():
    conf.set(k, v)

spark = (SparkSession.builder
         .enableHiveSupport()
         .config(conf=conf)
         .getOrCreate()
)
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
output_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/nvs_xolair_tables/nvs_tam_hd/'
today = datetime.today().strftime('%Y%m%d')
table_name = 'nvs_tam' 

#Tam
df_tam = spark.sql(
"""
    with normalized as(
         select 
            hcp.mdm_zip as zip_code,
            nvs.yrmo as year_month,
            'XOLAIR' as product_brand_name,
            case when call_p1 = '1' or lunch_n_learn_calls = '1' then 1 
            when call_p2 = '1' then 2
            when call_p3 = '1' then 3 
            end as display_order
        from sandbox_respiratory_commercial_analyst.mmix_nvs_xolair_calls_lle_raw_daily as nvs 
        inner join sandbox_measurement_analyst.mars_master_cm_mdm_profile_weekly_staging as mdm
        on nvs.npi_num = mdm.npi_number 
        inner join sandbox_measurement_analyst.mars_hcp_org_mapping_staging as hcp 
        on mdm.mdm_id = hcp.mdm_id and array_contains(hcp.product_brand_name, 'XOLAIR')
        where nvs.npi_num is not null
    ),
    tam_hd_costs as(
        select 'XOLAIR' as brand, 32000000.0 as cost, '2022' as year
        union all
        select 'XOLAIR' as brand, 32000000.0 as cost, '2023' as year
    ),
    hcp_costs as(
        select 
            n.product_brand_name, 
            n.year_month, 
            n.zip_code as zip,
            'CE' as audience, 
            'tam_hd' as channel, 
            1.0 / n.display_order as reach, 
            c.cost
        from normalized as n 
        inner join tam_hd_costs as c 
        on n.product_brand_name = c.brand and substring(n.year_month, 1, 4) = c.year 
    ),
    final_agg as(
        select 
            product_brand_name, 
            'NVS' as source,
            year_month, 
            zip,
            audience,
            channel, 
            reach,
            cast(null as double) as engage,
            cost * reach / sum(reach) over(partition by product_brand_name, substring(year_month, 1, 4)) as cost
        from hcp_costs 
    )
    select *
    from final_agg

"""
)
df_tam.registerTempTable('ce_tam')

df_tam.write.mode("overwrite").parquet(
    f"{output_path}{today}/"
)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS sandbox_measurement_analyst.mmix_{table_name}_staging
    USING PARQUET
    LOCATION '{output_path}{today}/'
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [ ]:
#display
df_display = spark.sql(
"""
    with cleaned_dmas as(
        select 
            distinct 
            dma_code, 
            dma_name from sandbox_measurement_analyst.mars_population_demographics_final
    ), 
    nvs_cleaned as(
        select 
            'XOLAIR' as brand, 
            'Display' as channel, 
            'DTC' as audience, 
            substring(year_mth, 1, 4) as year,
            substring(year_mth, 5, 2) as month,
            dma_code,
            cast(null as string) as zip_code, 
            cast(null as string) as state, 
            'US' as country, 
            sum(cast(impressions as double)) as reach, 
            sum(cast(clicks as double)) as engage
        from sandbox_respiratory_commercial_analyst.mmix_nvs_xolair_dtc_paid_display as nvs
        group by 1, 2, 3, 4, 5, 6
    ), 
    dma_joined as(
        select 
            brand, 
            channel, 
            audience, 
            month,
            year,
            zip_code, 
            cd.dma_name as dma,
            state, 
            country,
            reach,
            engage
        from nvs_cleaned as nvs 
        join cleaned_dmas as cd 
        on nvs.dma_code = cd.dma_code 
    ),
    costs_cleaned as(
        select 
            substring(date_month_, 1, 4) AS year,
            substring(date_month_, 6, 2) AS month,
            cast(replace(dtc_display_, ',', '') as double) as cost
        from sandbox_measurement_analyst.nvs_spend_processed
    ),
    final_agg as(
        select 
            brand, 
            channel, 
            audience, 
            dj.year,
            cast(dj.month as integer) as month,
            zip_code, 
            dma,
            state, 
            country,
            reach,
            engage,
            cast(cc.cost as double) * reach / sum(reach) over(partition by brand, dj.month, dj.year) as cost
        from dma_joined dj 
        inner join costs_cleaned cc 
        on dj.year = cc.year and dj.month = cc.month 
    )
    select 
        *
    from final_agg
"""
)
df_display.registerTempTable('dtc_display')
df_display.printSchema()

In [ ]:
#paid search 
df_paid_search = spark.sql(
"""
    with cleaned_dmas as(
        select 
            distinct 
            dma_code, 
            dma_name from sandbox_measurement_analyst.mars_population_demographics_final
    ),
    nvs_cleaned as(
        select 
            'XOLAIR' as brand, 
            'Paid Search' as channel, 
            'DTC' as audience, 
            substring(year_mth, 1, 4) as year,
            substring(year_mth, 5, 2) as month,
            dma_code,
            cast(null as string) as zip_code, 
            cast(null as string) as state, 
            'US' as country, 
            sum(cast(impressions as double)) as reach, 
            sum(cast(clicks as double)) as engage
        from sandbox_respiratory_commercial_analyst.mmix_nvs_xolair_dtc_paid_search as nvs
        group by 1, 2, 3, 4, 5, 6
    ),
    dma_joined as(
        select 
            brand, 
            channel, 
            audience, 
            month,
            year,
            zip_code, 
            cd.dma_name as dma,
            state, 
            country,
            reach,
            engage
        from nvs_cleaned as nvs 
        join cleaned_dmas as cd 
        on nvs.dma_code = cd.dma_code 
    ),
    costs_cleaned as(
        select 
            substring(date_month_, 1, 4) AS year,
            substring(date_month_, 6, 2) AS month,
            cast(replace(dtc_search, ',', '') as double) as cost
        from sandbox_measurement_analyst.nvs_spend_processed
    ),
    final_agg as(
        select 
            brand, 
            channel, 
            audience, 
            dj.year,
            cast(dj.month as integer) as month,
            zip_code, 
            dma,
            state, 
            country,
            reach,
            engage,
            cast(cc.cost as double) * reach / sum(reach) over(partition by brand, dj.month, dj.year) as cost
        from dma_joined dj 
        inner join costs_cleaned cc 
        on dj.year = cc.year and dj.month = cc.month 
    )
    select 
        *
    from final_agg
""")
df_paid_search.registerTempTable('dtc_paid_search')
df_paid_search.printSchema()

In [ ]:
#point of care
df_poc = spark.sql(
"""
    with cleaned_dmas as(
        select 
            distinct 
            dma_code, 
            dma_name from sandbox_measurement_analyst.mars_population_demographics_final
    ),
    nvs_cleaned as(
        select 
            'XOLAIR' as brand, 
            'Point of Care' as channel, 
            'DTC' as audience, 
            substring(year_mth, 1, 4) as year,
            substring(year_mth, 5, 2) as month,
            dma_code,
            cast(null as string) as zip_code, 
            cast(null as string) as state, 
            'US' as country, 
            sum(cast(impressions as double)) as reach, 
            cast(null as double) as engage
        from sandbox_respiratory_commercial_analyst.mmix_nvs_xolair_dtc_poc as nvs
        group by 1, 2, 3, 4, 5, 6, 7, 8, 9
    ),
    dma_joined as(
        select 
            brand, 
            channel, 
            audience, 
            month,
            year,
            zip_code, 
            cd.dma_name as dma,
            state, 
            country,
            reach,
            engage
        from nvs_cleaned as nvs 
        join cleaned_dmas as cd 
        on nvs.dma_code = cd.dma_code 
    ),
    costs_cleaned as(
        select 
            substring(date_month_, 1, 4) AS year,
            substring(date_month_, 6, 2) AS month,
            cast(replace(dtc_poc, ',', '') as double) as cost
        from sandbox_measurement_analyst.nvs_spend_processed
    ),
    final_agg as(
        select 
            brand, 
            channel, 
            audience, 
            dj.year,
            cast(dj.month as integer) as month,
            zip_code, 
            dma,
            state, 
            country,
            reach,
            engage,
            cast(cc.cost as double) * reach / sum(reach) over(partition by brand, dj.month, dj.year) as cost
        from dma_joined dj 
        inner join costs_cleaned cc 
        on dj.year = cc.year and dj.month = cc.month 
    )
    select 
        *
    from final_agg
""")
df_poc.registerTempTable('dtc_poc')
df_poc.printSchema()

In [ ]:
df_paid_social = spark.sql(
"""
    with cleaned_dmas as(
        select 
            distinct 
            dma_code, 
            dma_name from sandbox_measurement_analyst.mars_population_demographics_final
    ),

    nvs_cleaned as(
        select 
            'XOLAIR' as brand, 
            'Paid Social' as channel, 
            'DTC' as audience, 
            substring(year_mth, 1, 4) as year,
            substring(year_mth, 5, 2) as month,
            dma_code,
            cast(null as string) as zip_code, 
            cast(null as string) as state, 
            'US' as country, 
            sum(cast(impressions as double)) as reach, 
            sum(cast(clicks as double)) as engage
        from sandbox_measurement_analyst.nvs_dtc_social_staging as nvs
        group by 1, 2, 3, 4, 5, 6
    ),
    dma_joined as(
        select 
            brand, 
            channel, 
            audience, 
            month,
            year,
            zip_code, 
            cd.dma_name as dma,
            state, 
            country,
            reach,
            engage
        from nvs_cleaned as nvs 
        join cleaned_dmas as cd 
        on nvs.dma_code = cd.dma_code 
    ),

    costs_cleaned as(
        select 
            substring(date_month_, 1, 4) AS year,
            substring(date_month_, 6, 2) AS month,
            cast(replace(dtc_social, ',', '') as double) as cost
        from sandbox_measurement_analyst.nvs_spend_processed
    ),
    final_agg as(
        select 
            brand, 
            channel, 
            audience, 
            dj.year,
            cast(dj.month as integer) as month,
            zip_code, 
            dma,
            state, 
            country,
            reach,
            engage,
            cast(cc.cost as double) * reach / sum(reach) over(partition by brand, dj.month, dj.year) as cost
        from dma_joined dj 
        inner join costs_cleaned cc 
        on dj.year = cc.year and dj.month = cc.month 
    )
    select *
    from final_agg 
""")
df_paid_social.registerTempTable('dtc_paid_social')
df_paid_social.printSchema()

In [ ]:
df_paid_search_hcp = spark.sql(
"""
    with cleaned_dmas as(
        select 
            distinct 
            dma_code, 
            dma_name from sandbox_measurement_analyst.mars_population_demographics_final
    ),
    nvs_cleaned as (
        select 
            'XOLAIR' as brand, 
            'Paid Search' as channel, 
            'HCP' as audience, 
            year(date(activity_date)) as year, 
            month(date(activity_date)) as month,
            dma_code,
            cast(null as string) as zip_code, 
            cast(null as string) as state, 
            'US' as country, 
            sum(cast(impressions as double)) as reach, 
            sum(cast(clicks as double)) as engage
        from sandbox_respiratory_commercial_analyst.mmix_nvs_xolair_npp_paid_search as nvs
        where activity_date <= '2023-12-29'
        group by 1, 2, 3, 4, 5, 6
    ),
    dma_joined as(
        select 
            brand, 
            channel, 
            audience, 
            month,
            year,
            zip_code, 
            cd.dma_name as dma,
            state, 
            country,
            reach,
            engage
        from nvs_cleaned as nvs 
        join cleaned_dmas as cd 
        on nvs.dma_code = cd.dma_code 
    ),
    nvs_costs as (
          select 
            substring(date_month_, 1, 4) AS year,
            substring(date_month_, 6, 2) AS month,
            cast(replace(npp, ',', '') as double) as total_cost
        from sandbox_measurement_analyst.nvs_spend_processed
    ),
    cost_allocation as(
        select 
            avg(cast(total_cost * 0.16 as double)) as paid_search_cost_allocation 
        from nvs_costs 
    ),
    total_reach as (
        select sum(reach) as total_reach
        from dma_joined
    ),
    final_allocation as (
        select 
            nc.brand, 
            nc.channel, 
            nc.audience,
            cast(nc.year as string) as year,
            cast(nc.month as integer) as month, 
            nc.zip_code,
            nc.dma, 
            nc.state, 
            nc.country, 
            nc.reach,
            nc.engage,
            (nc.reach / tr.total_reach) * ca.paid_search_cost_allocation as cost
        from dma_joined nc
        cross join total_reach tr
        cross join cost_allocation ca
    )
    select 
       *
    from final_allocation
"""
)
df_paid_search_hcp.registerTempTable('hcp_paid_search')
df_paid_search_hcp.printSchema()

In [ ]:
df_all_channels = spark.sql(
"""
    with spend_data as (
        select 
            substring(date_month_, 1, 4) as year,
            substring(date_month_, 6, 2) as month,
            cast(replace(npp, ',', '') as double) as total_cost
        from sandbox_measurement_analyst.nvs_spend_processed
    ), 
    avg_allocations AS (
        select 
            avg(cast(total_cost * 0.39 as double)) as Display,
            avg(cast(total_cost * 0.22 as double)) as Paid_Social,
            avg(cast(total_cost * 0.06 as double)) as Third_Party_Email,
            avg(cast(total_cost * 0.08 as double)) as Point_of_Care,
            avg(cast(total_cost * 0.08 as double)) as Custom,
            avg(cast(total_cost * 0.01 as double)) as EHR_EMR
        from spend_data
    ),
    pivoted_hcp as(
        select 
            'XOLAIR' as brand,
            channel, 
            'HCP' as audience,
            cast(substring(yrwk, 1, 4) AS string) as year,
            cast(substring(yrwk, 5, 2) AS string) as week,
            zip_cd as zip_code,
            cast(null as string) as dma,
            cast(null as string) as state,
            'US' as country,
            sum(case when metric = 'REACH' then cast(value as double) end) as reach,
            sum(case when metric = 'ENGAGEMENT' then cast(value as double) end) as engage
        from sandbox_respiratory_commercial_analyst.mmix_nvs_xolair_npp_hcp_channels
        where zip_cd is not null
        group by 1, 2, 3, 4, 5, 6, 7, 8
    ),
    weekly_to_monthly as(
        select 
            brand, 
            case
                when channel like 'EHR' then 'EHR/EMR'
                when channel like '3RD_PARTY_EMAIL' then 'Third-Party Email'
                when channel like 'POC' then 'Point of Care'
                when channel like 'DISPLAY' then 'Display'
                when channel in ('VIDEO', 'CUSTOM') then 'Custom'
                when channel like 'ENDEMIC_SOCIAL' then 'Paid Social'
                end as channel,
            audience, 
            year,
            cast(date_format(add_months(to_date(concat(year, '-01-01'), 'yyyy-MM-dd'), (cast(week as int) - 1) / 4), 'MM') as int) AS month, 
            zip_code, 
            dma, 
            state, 
            country, 
            reach, 
            engage
        from pivoted_hcp p 
    ), 
    total_reach as (
        select channel, sum(reach) as total_reach
        from weekly_to_monthly
        group by channel
    ),
    final_allocation as (
        select 
            w.*,
            (w.reach / tr.total_reach) * 
            case 
                when w.channel = 'Display' then a.Display
                when w.channel = 'Paid Social' then a.Paid_Social
                when w.channel = 'Third-Party Email' then a.Third_Party_Email
                when w.channel = 'Point of Care' then a.Point_of_Care
                when w.channel = 'Custom' then a.Custom
                when w.channel = 'EHR/EMR' then a.EHR_EMR
            end as cost
        from weekly_to_monthly w
        cross join total_reach tr
        cross join avg_allocations a
        where w.channel = tr.channel
    )
    select * from final_allocation
""")

df_all_channels.registerTempTable('hcp_all_channels')
df_all_channels.printSchema()

In [ ]:
df_unioned = spark.sql(
"""
   with cte as(
        select * from dtc_display
        union all 
        select * from dtc_paid_search
        union all 
        select * from dtc_poc
        union all 
        select * from dtc_paid_social
        union all 
        select * from hcp_paid_search
        union all 
        select * from hcp_all_channels
 )
    select * from cte 
""")
df_unioned.registerTempTable('combined_nvs_data')

In [ ]:
output_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/nvs_xolair_tables/nvs_all_combined/'
today = datetime.today().strftime('%Y%m%d')
table_name = 'nvs_xo_combined' 

df_unioned.write.mode("overwrite").parquet(
    f"{output_path}{today}/"
)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS sandbox_measurement_analyst.mmix_{table_name}_staging
    USING PARQUET
    LOCATION '{output_path}{today}/'
""")

In [ ]:
#RUN XO CODE ON nvs_xo_combined BEFORE NEXT STEP. mmix_nvs_digital is the output table from XO code.

In [ ]:
#Just NVS Pivot Table:
nvs_pre_pivot = spark.sql(
    """
        select 
            product_brand_name,
            year_month, 
            zip as zip_code, 
            channel, 
            audience, 
            reach, 
            engage, 
            cost, 
            'NVS' as source
        from sandbox_measurement_analyst.mmix_nvs_digital
        where year_month between '202201' and '202312'
        union all 
        select 
            product_brand_name,
            year_month, 
            zip as zip_code, 
            channel,
            audience, 
            cast(reach as double) as reach, 
            cast(engage as double) as engage, 
            cast(cost as double) as cost, 
            'NVS' as source
        from sandbox_measurement_analyst.mmix_nvs_tam_staging
        where year_month between '202201' and '202312
    """    
)
nvs_pre_pivot.registerTempTable('nvs_pre_pivot')
nvs_pivoted = (
    spark.sql(
"""
    SELECT
        product_brand_name,
        zip_code,
        year_month,
        channel || '_' || audience AS channel_audience,
        reach,
        engage,
        cost
    FROM nvs_pre_pivot
"""
    )
    .groupBy(
        [
            "product_brand_name",
            "zip_code",
            "year_month"
        ]
    )
    .pivot("channel_audience")
    .sum("reach", "engage", "cost")
)

nvs_pivoted = nvs_pivoted.select(
    *[
        F.col(col).alias(col.replace("_sum(", "_").replace(")", ""))
        for col in nvs_pivoted.columns
    ]
)
nvs_pivoted.registerTempTable('nvs_post_pivot')
output_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/nvs_xolair_tables/nvs_all_pivoted/'
today = datetime.today().strftime('%Y%m%d')
table_name = 'nvs_pivoted'

nvs_pivoted.write.mode("overwrite").parquet(
    f"{output_path}{today}/"
)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS sandbox_measurement_analyst.mmix_{table_name}
    USING PARQUET
    LOCATION '{output_path}{today}/'
""")

In [ ]:
#NVS - GENE Combined
nvs_gene_pre_pivot = spark.sql(
    """
        select 
            product_brand_name,
            year_month, 
            zip as zip_code, 
            channel, 
            audience, 
            reach, 
            engage, 
            cost, 
            'NVS' as source
        from sandbox_measurement_analyst.mmix_nvs_digital
        where year_month between '202201' and '202312'
        union all 
        select 
            product_brand_name,
            year_month, 
            zip as zip_code, 
            channel,
            audience, 
            cast(reach as double) as reach, 
            cast(engage as double) as engage, 
            cast(cost as double) as cost, 
            'NVS' as source
        from sandbox_measurement_analyst.mmix_nvs_tam_staging
        where year_month between '202201' and '202312'
        union all 
        select
            product_brand_name,
            year_month,
            zip_code,
            channel,
            audience,
            cast(reach as double) as reach,
            cast(engage as double) as engage,
            cast(cost as double) as cost,
            'GENE' as source
        from sandbox_measurement_analyst.mars_pre_pivot_monthly
        where product_brand_name = 'XOLAIR' and year_month between '202201' AND '202312'
    """    
)
nvs_gene_pre_pivot.registerTempTable('nvs_gene_pre_pivot')
nvs_gene_pivoted = (
    spark.sql(
"""
    SELECT
        product_brand_name,
        zip_code,
        year_month,
        channel || '_' || audience AS channel_audience,
        reach,
        engage,
        cost
    FROM nvs_gene_pre_pivot
"""
    )
    .groupBy(
        [
            "product_brand_name",
            "zip_code",
            "year_month"
        ]
    )
    .pivot("channel_audience")
    .sum("reach", "engage", "cost")
)

nvs_gene_pivoted = nvs_gene_pivoted.select(
    *[
        F.col(col).alias(col.replace("_sum(", "_").replace(")", ""))
        for col in nvs_gene_pivoted.columns
    ]
)
nvs_gene_pivoted.registerTempTable('nvs_gene_pivoted')

nvs_gene_pivoted_final = spark.sql(
"""
    with cte as (
        select distinct
            coalesce(
                pivoted.product_brand_name,
                sales.product_brand_name,
                hcp_count.product_brand_name
            ) as product_brand_name,
            coalesce(pivoted.zip_code, sales.zip, hcp_count.zip_code) as zip_code,
            coalesce(pivoted.year_month, sales.year_month) as date_year_month, 
          custom_hcp_reach , 
          custom_hcp_engage , 
          custom_hcp_cost , 
          display_dtc_reach , 
          display_dtc_engage , 
          display_dtc_cost , 
          display_hcp_reach , 
          display_hcp_engage , 
          display_hcp_cost , 
          ehr_emr_hcp_reach , 
          ehr_emr_hcp_engage , 
          ehr_emr_hcp_cost , 
          enewsletter_hcp_reach , 
          enewsletter_hcp_engage , 
          enewsletter_hcp_cost , 
          house_email_dtc_reach , 
          house_email_dtc_engage , 
          house_email_dtc_cost , 
          house_email_hcp_reach , 
          house_email_hcp_engage , 
          house_email_hcp_cost , 
          journals_print_dtc_reach , 
          journals_print_dtc_engage , 
          journals_print_dtc_cost , 
          online_video_dtc_reach , 
          online_video_dtc_engage , 
          online_video_dtc_cost , 
          paid_search_dtc_reach , 
          paid_search_dtc_engage , 
          paid_search_dtc_cost , 
          paid_search_hcp_reach , 
          paid_search_hcp_engage , 
          paid_search_hcp_cost , 
          paid_social_dtc_reach , 
          paid_social_dtc_engage , 
          paid_social_dtc_cost , 
          paid_social_hcp_reach , 
          paid_social_hcp_engage , 
          paid_social_hcp_cost , 
          point_of_care_dtc_reach , 
          point_of_care_dtc_engage , 
          point_of_care_dtc_cost , 
          point_of_care_hcp_reach , 
          point_of_care_hcp_engage , 
          point_of_care_hcp_cost , 
          speaker_program_hcp_reach , 
          speaker_program_hcp_engage , 
          speaker_program_hcp_cost , 
          tam_hd_ce_reach , 
          tam_hd_ce_engage , 
          tam_hd_ce_cost , 
          third_party_email_hcp_reach , 
          third_party_email_hcp_engage , 
          third_party_email_hcp_cost , 
          video_dtc_reach , 
          video_dtc_engage , 
          video_dtc_cost, 
          sales.sales_qty as gne_867_sales_qty_eqv,
          hcp_count.hcp_count
        from nvs_gene_pivoted as pivoted
        full outer join
            sandbox_measurement_analyst.mars_sales_monthly_staging as sales
            on
                pivoted.product_brand_name = sales.product_brand_name
                and pivoted.year_month = sales.year_month
                and pivoted.zip_code = sales.zip
        full outer join
            sandbox_measurement_analyst.mars_hcp_count_staging as hcp_count
            on
                pivoted.product_brand_name = hcp_count.product_brand_name
                and pivoted.zip_code = hcp_count.zip_code
    )
    select * from cte 
    where date_year_month is not null
""")
nvs_gene_pivoted_final.registerTempTable('nvs_gene_pivoted_final')

nvs_gene_pivoted_to_athena = spark.sql("""select * from nvs_gene_pivoted_final where product_brand_name = 'XOLAIR'
                                                and date_year_month between '202201' and '202312'""")

output_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/nvs_xolair_tables/nvs_gene_pivoted/'
table_name = 'nvs_gene_pivoted'
today = datetime.today().strftime('%Y%m%d')

nvs_gene_pivoted_to_athena.write.mode("overwrite").parquet(
    f"{output_path}{today}/"
)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS sandbox_measurement_analyst.mmix_{table_name}
    USING PARQUET
    LOCATION '{output_path}{today}/'
""")